In [ ]:
import pandas as pd
import numpy as np
from scipy.special import gamma
from scipy.integrate import quad
from concurrent.futures import ProcessPoolExecutor
def gsnd_pdf(x, mu, sigma, b, r):
    epsilon = 1e-8  
    coeff = b / (2**(1+1/b) * gamma(1 / b))
    exponent = -np.abs(x - mu) ** b / (2 * sigma ** b * (1 + r * np.sign(x - mu) + epsilon) ** b)
    return coeff / sigma * np.exp(exponent)
def mixed_pdf(x, components):
    pdf_val = 0
    for weight, mu, sigma, b, r in components:
        pdf_val += weight * gsnd_pdf(x, mu, sigma, b, r)
    return pdf_val
def mixed_cdf(x_vals, components, integration_range):
    cdf_vals = []
    for x in x_vals:
        lower_bound = -integration_range
        try:
            result, _ = quad(mixed_pdf, lower_bound, x, args=(components,), epsabs=1e-6, epsrel=1e-6)
            cdf_vals.append(result)
        except Exception:
            cdf_vals.append(0)  
    return np.array(cdf_vals)
def wasserstein_distance_mixed(components1, components2, integration_range, resolution=1000):
    lower_bound = -integration_range
    upper_bound = integration_range
    x_vals = np.linspace(lower_bound, upper_bound, resolution)

    cdf1 = mixed_cdf(x_vals, components1, integration_range)
    cdf2 = mixed_cdf(x_vals, components2, integration_range)

    return np.trapezoid(np.abs(cdf1 - cdf2), x_vals)